In [59]:
##### Your Code Here #####
import re
import string

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.probability import FreqDist

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Now it's your turn!

Use the following dataset of scraped "Data Scientist" and "Data Analyst" job listings to create your own Document Classification Models.

<https://raw.githubusercontent.com/LambdaSchool/DS-Unit-4-Sprint-2-NLP/master/module3-Document-Classification/job_listings.csv>

Requirements:

- Apply both CountVectorizer and TfidfVectorizer methods to this data and compare results
- Use at least two different classification models to compare differences in model accuracy
- Try to "Hyperparameter Tune" your model by using different n_gram ranges, max_results, and data cleaning methods
- Try and get the highest accuracy possible!

### Vectorizing!

In [0]:
scientists_url = 'https://raw.githubusercontent.com/zarrinan/DS-Unit-4-Sprint-2-NLP/master/module3-Document-Classification/df_scientist.csv'
analysts_url = 'https://raw.githubusercontent.com/zarrinan/DS-Unit-4-Sprint-2-NLP/master/module3-Document-Classification/df_analyst.csv'
url = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-4-Sprint-2-NLP/master/module3-Document-Classification/job_listings.csv'

In [0]:
pd.set_option('display.max_colwidth', 200)

In [0]:
df_a = pd.read_csv(analysts_url, encoding="ISO-8859-1")
df_s = pd.read_csv(scientists_url, encoding="ISO-8859-1")
df =  pd.read_csv(url, encoding="ISO-8859-1")

In [0]:
df['desc'] = [BeautifulSoup(str(item), 'html.parser').get_text()[2:] for item in df['description']]
df.drop(['description'], axis=1, inplace=True)
df['if_scientist'] = df['job'].str.contains('Scientist')
df = shuffle(df)
df.head()

,title,job,desc,if_scientist
438,Data Analyst (Mgmt Systems Analyst),Data Analyst,"$34.76 - $40.48 an hourMission\nTo connect people with resources and solutions in times of need so we can all live, learn, work and take part in strong, healthy communities\n\nVision\nThat all bas...",False
261,Data Analyst,Data Analyst,The Role\nCome help our customers learn more about the world. Below is a small sample of the questions we\'ve asked of our data over the last year.\n\nDid Lyft\'s prospects improve in politically ...,False
247,Senior Data Scientist (GEC11902),Data Scientist,Position Description\nyou need to have\nPhD in computer science or similar field with 3+ years of experienceHighly proficient in Python or JavaExtremely hands on in building Machine Learning produ...,True
473,BrightStart - Intern - Salesforce Data Analyst,Data Analyst,"InternshipBrightStart Intern - Salesforce Data Analyst\nUndergraduate\nLocation: Lawrenceville, GA\nOur team:\nAt Brightree, we believe in building innovative technology solutions and providing se...",False
157,Investigative Data Scientist,Data Scientist,"Channel and Intellectual Property Protection Services Investigative Data Scientist\n\nCome as you are. Do what you love.\n\nMicrosoft a global company, located in over 100 countries, with revenues...",True


In [0]:
def clean_preprocess_words(words_list):
  cleaned_list = []
  for word in words_list:
    #Tokenize by word
    tokens = word_tokenize(word)
    #print('Tokens:', tokens)

    #Make all words lowercase
    lower_tokens = [w.lower() for w in tokens]
    #print(lower_tokens)

    #strip punctuation from within words
    no_punctuation = [w.translate(table) for w in lower_tokens]
    #print(no_punctuation)

    #remove words that aren't alphabetic
    alphabetic = [word for word in no_punctuation if word.isalpha()]
    #print(alphabetic)

    #remove stopwords
    words = [w for w in alphabetic if not w in stop_words]
    #print(words)

    #append to cleaned list
    cleaned_list.append(words)
    
  return cleaned_list

In [0]:
desc_a = df_a.description
desc_s = df_s.description
desc = df.desc

In [0]:
def vectorize(df):
  from sklearn.feature_extraction.text import CountVectorizer
  vectorizer = CountVectorizer(max_features=None, ngram_range=(1,2), stop_words='english')
  vectorizer.fit(df)
  print(vectorizer.vocabulary_)

In [0]:
def tfidf(df):
  from sklearn.feature_extraction.text import TfidfVectorizer
  vectorizer = TfidfVectorizer(max_features=None, ngram_range=(1,2), stop_words='english')
  vectorizer.fit(df)
  print(vectorizer.vocabulary_)

### Classification!

In [50]:
#split the data for joined df
def split_data(df, target):
  from sklearn.model_selection import train_test_split
  X, y = df.description, df[target]
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  return X_train, X_test, y_train, y_test

#join data scientists and data analysts dataframes
data_joined = df_a.append(df_s)
data_joined['scientists'] = data_joined['title'].str.contains('Scientist')


#shuffle the joined dataframe and select only description and title columns
from sklearn.utils import shuffle
data_joined = shuffle(data_joined)
data_joined_ref = data_joined[['description', 'scientists']]


#separate data into training and test
X_train, X_test, y_train, y_test = split_data(data_joined_ref, 'scientists')
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(288,)
(72,)
(288,)
(72,)


In [63]:
#split the data for the df
def split_data(df, target):
  from sklearn.model_selection import train_test_split
  X, y = df.desc, df[target]
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  return X_train, X_test, y_train, y_test



#separate data into training and test
X_train, X_test, y_train, y_test = split_data(df, 'if_scientist')
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(400,)
(100,)
(400,)
(100,)


#### Count Vectorizer

In [55]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=None, ngram_range=(1,1), stop_words='english')

vectorizer.fit(X_train)

print(vectorizer.vocabulary_)

{'day': 2089, 'passionate': 6861, 'employees': 2672, 'connect': 1732, 'customers': 2027, 'world': 9611, 'xe2': 9748, 'x80': 9632, 'x93': 9645, 'x99s': 9675, 'exist': 2906, 'shaw': 8216, 'strive': 8632, 'place': 7026, 'best': 979, 'people': 6907, 'choose': 1409, 'work': 9591, 'reflect': 7632, 'richness': 7885, 'communities': 1599, 'culture': 2009, 'inclusive': 3849, 'individual': 3888, 'diverse': 2418, 'background': 897, 'perspective': 6960, 'makes': 4587, 'stronger': 8637, 'team': 8842, 'nat': 5134, 'believe': 963, 'workforce': 9600, 'fosters': 3218, 'diversity': 2420, 'thought': 8957, 'means': 4704, 'solutions': 8377, 'invite': 4157, 'qualified': 7453, 'individuals': 3891, 'apply': 662, 'nthis': 6411, 'position': 7101, 'reports': 7741, 'dan': 2046, 'lavender': 4346, 'ncareer': 5232, 'opportunity': 6691, 'data': 2055, 'scientist': 8054, 'nshaw': 6296, 'communications': 1595, 'exciting': 2885, 'consumer': 1787, 'deep': 2130, 'insights': 3983, 'reporting': 7739, 'director': 2329, 'succes

In [56]:
train_word_counts = vectorizer.transform(X_train)

X_train_vectorized = pd.DataFrame(train_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_train_vectorized.shape)
X_train_vectorized.head()

(400, 9793)


,00,000,00011236,00805,00pm,01,02115,03,04,05,...,zero,zetahub,zeus,zheng,zillow,zoho,zone,zones,zoom,zywave
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
test_word_counts = vectorizer.transform(X_test)

X_test_vectorized = pd.DataFrame(test_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_test_vectorized.shape)
X_test_vectorized.head()

(100, 9793)


,00,000,00011236,00805,00pm,01,02115,03,04,05,...,zero,zetahub,zeus,zheng,zillow,zoho,zone,zones,zoom,zywave
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
#Multinomial Naive Bayes Model on Count Vectorized data
from sklearn.naive_bayes import MultinomialNB
MNB = MultinomialNB().fit(X_train_vectorized, y_train)
train_predictions = MNB.predict(X_train_vectorized)
test_predictions = MNB.predict(X_test_vectorized)
print(f'Train Roc-Auc: {roc_auc_score(y_train, train_predictions)}')
print(f'Test Roc-Auc: {roc_auc_score(y_test, test_predictions)}')

Train Roc-Auc: 0.9648859543817527
Test Roc-Auc: 0.8900966183574879


In [62]:
#XGBoostClassifier Model on Count Vectorized data

from xgboost.sklearn import XGBClassifier
XGB = XGBClassifier(n_estimators=200, num_class=len(df.job.unique()), objective='multi:softmax').fit(X_train_vectorized, y_train)
train_predictions = XGB.predict(X_train_vectorized)
test_predictions = XGB.predict(X_test_vectorized)
print(f'Train Roc-Auc: {roc_auc_score(y_train, train_predictions)}')
print(f'Test Roc-Auc: {roc_auc_score(y_test, test_predictions)}')

Train Roc-Auc: 0.992547018807523
Test Roc-Auc: 1.0


### TF-IDF

In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=None, ngram_range=(1,1), stop_words='english')

vectorizer.fit(X_train)

train_word_counts = vectorizer.transform(X_train)

X_train_vectorized = pd.DataFrame(train_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_train_vectorized.shape)
X_train_vectorized.head()


(400, 9793)


,00,000,00011236,00805,00pm,01,02115,03,04,05,...,zero,zetahub,zeus,zheng,zillow,zoho,zone,zones,zoom,zywave
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
test_word_counts = vectorizer.transform(X_test)

X_test_vectorized = pd.DataFrame(test_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_test_vectorized.shape)
X_test_vectorized.head()

(100, 9793)


,00,000,00011236,00805,00pm,01,02115,03,04,05,...,zero,zetahub,zeus,zheng,zillow,zoho,zone,zones,zoom,zywave
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
#Multinomial Naive Bayes Model on TF-IDF Vectorized data

from sklearn.naive_bayes import MultinomialNB
MNB = MultinomialNB().fit(X_train_vectorized, y_train)
train_predictions = MNB.predict(X_train_vectorized)
test_predictions = MNB.predict(X_test_vectorized)
print(f'Train Roc-Auc: {roc_auc_score(y_train, train_predictions)}')
print(f'Test Roc-Auc: {roc_auc_score(y_test, test_predictions)}')

Train Roc-Auc: 0.9569327731092437
Test Roc-Auc: 0.8623188405797102


In [67]:
#XGBoostClassifier Model on TF-IDF Vectorized data

from xgboost.sklearn import XGBClassifier
XGB = XGBClassifier(n_estimators=200, num_class=len(df.job.unique()), objective='multi:softmax').fit(X_train_vectorized, y_train)
train_predictions = XGB.predict(X_train_vectorized)
test_predictions = XGB.predict(X_test_vectorized)
print(f'Train Roc-Auc: {roc_auc_score(y_train, train_predictions)}')
print(f'Test Roc-Auc: {roc_auc_score(y_test, test_predictions)}')

Train Roc-Auc: 0.9926470588235294
Test Roc-Auc: 1.0


# Stretch Goals

- Try some agglomerative clustering using cosine-similarity-distance! (works better with high dimensional spaces) robust clustering - Agglomerative clustering like Ward would be cool. Try and create an awesome Dendrogram of the most important terms from the dataset.

- Awesome resource for clustering stretch goals: 
 - Agglomerative Clustering with Scipy: <https://joernhees.de/blog/2015/08/26/scipy-hierarchical-clustering-and-dendrogram-tutorial/>
 - Agglomerative Clustering for NLP: <http://brandonrose.org/clustering>
 
- Use Latent Dirichlet Allocation (LDA) to perform topic modeling on the dataset: 
 - Topic Modeling and LDA in Python: <https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24>
 - Topic Modeling and LDA using Gensim: <https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/>
